In [52]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

### 定義目標函數

In [53]:
def fitness(x1, x2):
    u = (1+(x1+x2+1)**2*(19-14*x1+3*x1**2-14*x2+6*x1*x2+3*x2**2))
    v = (30+(2*x1-3*x2)**2*(18-32*x1+12*x1**2+48*x2-36*x1*x2+27*x2**2))
    return u*v

### 定義 Simplex Direct Search

In [54]:
n = 2
def simplex(init_sol, n = n, edge_length = 2):
    sx_sol = []
    if init_sol[0] + edge_length > 2:
        sx_sol.append([round(init_sol[0] - edge_length, 4), init_sol[1]])
    else:
        sx_sol.append([round(init_sol[0] + edge_length, 4), init_sol[1]])
    if init_sol[1] + edge_length > 2:
        sx_sol.append([init_sol[0], round(init_sol[1] - edge_length, 4)])
    else:
        sx_sol.append([init_sol[0], round(init_sol[1] + edge_length, 4)])
    return sx_sol

### 計算重心

In [55]:
def centroid(dataset):
    x1_sum = 0; x2_sum = 0
    for i in range(len(dataset)):
        x1_sum += dataset[i][0] 
        x2_sum += dataset[i][1]
    return round(x1_sum/(len(dataset)), 4), round(x2_sum/(len(dataset)), 4)

### 參數設定

In [64]:
# step0 Setting parameters
np.random.seed(11)
# 變數數量、每個溫度迭代次數

n = 2
M = n
MaxIter = 50*n

# sigma & accepted probability & termination criteria
sigma = np.random.rand()
p = 0.9                    #initial
epsilon = 1e-8 

# 溫度衰減係數
alpha = 0.5 # 調低 加速降溫

# 初始解
curx = [round(random.uniform(-2, 2), 4), round(random.uniform(-2, 2), 4)]
print("初始解：", curx)

# BEST Fit
BestFit = []
maxNumFit = 2*n

# Simplex Direct Search
S = []; SortedS=[]
S.append(curx)
S.extend(simplex(curx))
print("Original solution set: %s" %S)
fit_list = []
for i1 in range(len(S)):
    x1 = S[i1][0]; x2 = S[i1][1]
    fit_list.append([i1, fitness(x1, x2)])

初始解： [-1.5712, 0.4251]
Original solution set: [[-1.5712, 0.4251], [0.4288, 0.4251], [-1.5712, -1.5749]]


In [65]:
import time
tStart = time.time() #計時開始

# step1: Order & re-label
for j1 in range(len(S)):
    SortedS.append(S[sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][0]])
    # store best fit
    BestFit.append([S[sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][0]], sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][1]])
if len(BestFit) > maxNumFit:
    del BestFit[maxNumFit:]
print('最佳組合 %s' %BestFit )
curx = SortedS[0]
minfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[0][1]
maxfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[-1][1]



# 初始溫度、溫度下限
maxT = (maxfit-minfit)/np.exp(p) # initial p = 0.9
curT = maxT
minT = 1e-5 * maxT
print('初始溫度: %.f & 終止溫度: %.f' %(curT, minT))

print('初始解(x1): %s & 初始目標函數值(f[x1]): %.f' %(curx, minfit))

#print(SortedS)
# step2
iters = 1
while ((maxfit-minfit) >= epsilon and curT > minT) or iters <= MaxIter:
    # step3
    for epoch in range(M):
        #step3.1
        k = 1
        print(SortedS)
        #step3.2
        while k <= n:
            #step3.3 Reflect
            #print(SortedS[:n-k+1])
            center = centroid(SortedS[:n-k+1])
            #print('重心:', center)
            x1 = SortedS[-k][0]; x2 = SortedS[-k][1]
            ρ_ = random.uniform(0.9, 1.1) 
            #print(ρ_)
            if round(center[0] + ρ_*(center[0] - x1),4) > 2 or round(center[0] + ρ_*(center[0] - x1),4) < -2:
                x1_r = round(center[0] - ρ_*(center[0] - x1),4)
            else:
                x1_r = round(center[0] + ρ_*(center[0] - x1),4)
            if round(center[1] + ρ_*(center[1] - x2),4) > 2 or round(center[1] + ρ_*(center[1] - x2),4) < -2:
                x2_r = round(center[1] - ρ_*(center[1] - x2),4)
            else:
                x2_r = round(center[1] + ρ_*(center[1] - x2),4)  
                
            #print('Before Reflect :', x1, x2)
            #print('Reflected :', x1_r, x2_r)
            f_min = fitness(x1_r, x2_r)
            #print('New fitness %.f'%f_min)
            #step3.3.1
            if f_min < minfit:
                #step3.3.3
                SortedS[-k] = [x1_r, x2_r]
                #minfit = f_min
                # step3.5
                fit_list_1 = []
                for i2 in range(len(SortedS)):
                    x1 = SortedS[i2][0]; x2 = SortedS[i2][1]
                    fit_list_1.append([i2, fitness(x1, x2)])
                S_ = []    
                for j2 in range(len(SortedS)):
                    S_.append(SortedS[sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[j2][0]])
                    
                # update best fit
                print('-'*10,'Doing insert','-'*10)
                BestFit.insert(0,[[x1_r, x2_r], f_min])
                if len(BestFit) > maxNumFit:
                    del BestFit[maxNumFit:]
                print('最佳組合 %s' %BestFit )
                
                curx = S_[0]
                SortedS = S_
                minfit = sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[0][1]
                maxfit = sorted(fit_list_1,key = lambda fit_list_1:fit_list_1[1])[-1][1]
                print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                #epoch += 1
                break
            #step3.3.2
            else:
                p = np.exp(-(f_min-minfit)/curT)
                #print('Prob: %.2f' %p)
                if p >= np.random.rand():
                    SortedS[-k] = [x1_r, x2_r]
                    fit_list_2 = []
                    for i3 in range(len(SortedS)):
                        x1 = SortedS[i3][0]; x2 = SortedS[i3][1]
                        fit_list_2.append([i3, fitness(x1, x2)])
                    #print(SortedS)
                    S_ = []    
                    for j3 in range(len(SortedS)):
                        S_.append(SortedS[sorted(fit_list_2,key = lambda fit_list_2: fit_list_2[1])[j3][0]])
                    # update best fit
                    loc = 0
                    for idx in range(len(BestFit)):
                        if f_min > BestFit[idx][1]:
                            loc = idx+1
                    print('-'*10,'Doing insert','-'*10)
                    BestFit.insert(loc,[[x1_r, x2_r], f_min])
                    if len(BestFit) > maxNumFit:
                        del BestFit[maxNumFit:]
                    print('最佳組合 %s' %BestFit )
                    
                    curx = S_[0]
                    SortedS = S_
                    minfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[0][1]
                    maxfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[-1][1]
                    print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                    #epoch += 1
                    break
                else:
                    print('K %s' %k)
                    k += 1
            

        print('-'*10, 'end of epoch %s' %epoch, '-'*10)                            
        epoch += 1        
    curT *= alpha    
    iters += 1
    print('-'*10,'current temperature %.f & iterations %.f & fitness = %.f' %(curT, iters, minfit), '-'*10)
print('最終收斂的解與解的品質:' , curx, minfit)
tEnd = time.time()#計時結束
print ("It cost %f sec" % (tEnd - tStart))

最佳組合 [[[0.4288, 0.4251], 1125.9372753603166], [[-1.5712, 0.4251], 5319.470058150081], [[-1.5712, -1.5749], 13052.184129791707]]
初始溫度: 4849 & 終止溫度: 0
初始解(x1): [0.4288, 0.4251] & 初始目標函數值(f[x1]): 1126
[[0.4288, 0.4251], [-1.5712, 0.4251], [-1.5712, -1.5749]]
---------- Doing insert ----------
最佳組合 [[[0.4288, 0.4251], 1125.9372753603166], [[0.4584, -1.634], 2057.704388541241], [[-1.5712, 0.4251], 5319.470058150081], [[-1.5712, -1.5749], 13052.184129791707]]
當下最佳解: [0.4288, 0.4251] & fitness: 1126
---------- end of epoch 0 ----------
[[0.4288, 0.4251], [0.4584, -1.634], [-1.5712, 0.4251]]
K 1
---------- Doing insert ----------
最佳組合 [[[0.4007, -1.5322], 976.3135021882365], [[0.4288, 0.4251], 1125.9372753603166], [[0.4584, -1.634], 2057.704388541241], [[-1.5712, 0.4251], 5319.470058150081]]
當下最佳解: [0.4007, -1.5322] & fitness: 976
---------- end of epoch 1 ----------
---------- current temperature 2424 & iterations 2 & fitness = 976 ----------
[[0.4007, -1.5322], [0.4288, 0.4251], [-1.5712, 0.

In [62]:
print(BestFit)

[[[0.4266, -0.0923], 942.4757770365594], [[0.4266, -1.54], 952.2853528618571], [[0.4266, 0.2418], 955.3800767552406], [[0.4266, 0.1422], 981.7202381179064]]


In [71]:
MaxIter = 50*n
epsilon2 = 1e-6 
for idx in range(len(BestFit)):
    # Simplex Direct Search
    S = []; SortedS=[]; curx = BestFit[idx][0]
    S.append(curx)
    S.extend(simplex(curx))



    # step1: Order & re-label
    for j1 in range(len(S)):
        SortedS.append(S[sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][0]])
    #print(len(SortedS))
    curx = SortedS[0]
    minfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[0][1]
    maxfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[-1][1]

    #print('初始解(x1): %s & 初始目標函數值(f[x1]): %.f' %(curx, minfit))

    # step2
    iters = 1
    while (maxfit-minfit) >= epsilon2 and iters <= MaxIter:
        k = 1
        #print(SortedS)
        while k <= n:
            shrink = True
            #step3.3 Reflect
            #print(SortedS[:n-k+1])
            center = centroid(SortedS[:n-k+1])
            #print('重心:', center)
            x1 = SortedS[-k][0]; x2 = SortedS[-k][1]
            ρ_ = random.uniform(0.9, 1.1) 
            #print(ρ_)
            if round(center[0] + ρ_*(center[0] - x1),4) > 2 or round(center[0] + ρ_*(center[0] - x1),4) < -2:
                x1_r = round(center[0] - ρ_*(center[0] - x1),4)
            else:
                x1_r = round(center[0] + ρ_*(center[0] - x1),4)
            if round(center[1] + ρ_*(center[1] - x2),4) > 2 or round(center[1] + ρ_*(center[1] - x2),4) < -2:
                x2_r = round(center[1] - ρ_*(center[1] - x2),4)
            else:
                x2_r = round(center[1] + ρ_*(center[1] - x2),4)  

                #print('Before Reflect :', x1, x2)
                #print('Reflected :', x1_r, x2_r)
            f_min = fitness(x1_r, x2_r)
                #print('New fitness %.f'%f_min)
                #step3.3.1
            if f_min < minfit:
                    #step3.3.3
                SortedS[-k] = [x1_r, x2_r]
                    #minfit = f_min
                    # step3.5
                fit_list_1 = []
                for i2 in range(len(SortedS)):
                    x1 = SortedS[i2][0]; x2 = SortedS[i2][1]
                    fit_list_1.append([i2, fitness(x1, x2)])
                S_ = []    
                for j2 in range(len(SortedS)):
                    S_.append(SortedS[sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[j2][0]])
                curx = S_[0]
                SortedS = S_
                minfit = sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[0][1]
                maxfit = sorted(fit_list_1,key = lambda fit_list_1:fit_list_1[1])[-1][1]
                #print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                shrink = False
                iters += 1
                break
                #step3.3.2
            else:
                p = np.exp(-(f_min-minfit)/curT)
                #print('Prob: %.2f' %p)
                if p >= np.random.rand():
                    SortedS[-k] = [x1_r, x2_r]
                    fit_list_2 = []
                    for i3 in range(len(SortedS)):
                        x1 = SortedS[i3][0]; x2 = SortedS[i3][1]
                        fit_list_2.append([i3, fitness(x1, x2)])
                    #print(SortedS)
                    S_ = []    
                    for j3 in range(len(SortedS)):
                        S_.append(SortedS[sorted(fit_list_2,key = lambda fit_list_2: fit_list_2[1])[j3][0]])
                    curx = S_[0]
                    SortedS = S_
                    minfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[0][1]
                    maxfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[-1][1]
                    #print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                    shrink = False
                    iters += 1
                    break
                else:
                    #print('K %s' %k)
                    k += 1


        # shrink
        if shrink:            
            #print('Before shrink %s' % SortedS)
            S = [SortedS[0]]
            for val in range(1,3):
                x1 = SortedS[-val][0]; x2 = SortedS[-val][1]
                x1_s = round(SortedS[0][0] + sigma*(x1 - SortedS[0][0]), 4)
                x2_s = round(SortedS[0][1] + sigma*(x2 - SortedS[0][1]), 4)
                S.append([x1_s, x2_s])

            #print('After shrink %s' % S)

                # 計算 排序
            fit_list_3 = []
            for i4 in range(len(S)):
                x1 = S[i4][0]; x2 = S[i4][1]
                fit_list_3.append([i4, fitness(x1, x2)])
            SortedS = []    
            for j4 in range(len(S)):
                SortedS.append(S[sorted(fit_list_3,key = lambda fit_list_3: fit_list_3[1])[j4][0]])
            curx = SortedS[0]
            minfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[0][1]
            maxfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[-1][1]
            #print('當下最佳解: %s & fitness: %.f' %(curx, minfit))


        #print('-'*10, 'end of iteration %s' %iters, '-'*10)                                
        iters += 1
    print('最終收斂的解與解的品質:' , curx, minfit)


最終收斂的解與解的品質: [0.0, -1.0] 3.0
最終收斂的解與解的品質: [-0.0018, -1.0015] 3.001207463327922
最終收斂的解與解的品質: [0.628, -0.5855] 63.14466981303254
最終收斂的解與解的品質: [0.7618, -0.4777] 80.23483919183452


## De Joung function (DJ)

In [159]:
#De Joung function (DJ)
def fitness(x1, x2, x3):
    return x1**2+x2**2+x3**2

In [160]:
def simplex(init_sol, n = n, edge_length = 5):
    sx_sol = []
    if init_sol[0] + edge_length > 5:
        sx_sol.append([round(init_sol[0] - edge_length, 4), init_sol[1], init_sol[2]])
    else:
        sx_sol.append([round(init_sol[0] + edge_length, 4), init_sol[1], init_sol[2]])
    if init_sol[1] + edge_length > 5:
        sx_sol.append([init_sol[0], round(init_sol[1] - edge_length, 4), init_sol[2]])
    else:
        sx_sol.append([init_sol[0], round(init_sol[1] + edge_length, 4), init_sol[2]])
    if init_sol[2] + edge_length > 5:
        sx_sol.append([init_sol[0], init_sol[1], round(init_sol[2] - edge_length, 4)])
    else:
        sx_sol.append([init_sol[0], init_sol[1], round(init_sol[2] + edge_length, 4)])
    return sx_sol

In [161]:
def centroid(dataset):
    x1_sum = 0; x2_sum = 0; x3_sum = 0
    for i in range(len(dataset)):
        x1_sum += dataset[i][0] 
        x2_sum += dataset[i][1]
        x3_sum += dataset[i][2]
    return round(x1_sum/(len(dataset)), 4), round(x2_sum/(len(dataset)), 4), round(x3_sum/(len(dataset)), 4)

In [162]:
# step0 Setting parameters
np.random.seed(11)
# 變數數量、每個溫度迭代次數

n = 3
M = n
MaxIter = 50*n

# sigma & accepted probability & termination criteria
sigma = np.random.rand()
p = 0.9                    #initial
epsilon = 1e-8 

# 溫度衰減係數
alpha = 0.5 # 調低 加速降溫

# 初始解
curx = [round(random.uniform(-5, 5), 4), round(random.uniform(-5, 5), 4), round(random.uniform(-5, 5), 4)]
print("初始解：", curx)

# BEST Fit
BestFit = []
maxNumFit = 2*n

# Simplex Direct Search
S = []; SortedS=[]
S.append(curx)
S.extend(simplex(curx, edge_length = 0.25))
print("Original solution set: %s" %S)
fit_list = []
for i1 in range(len(S)):
    x1 = S[i1][0]; x2 = S[i1][1]; x3 = S[i1][2]
    fit_list.append([i1, fitness(x1, x2, x3)])

初始解： [-4.1206, 0.8143, -0.7775]
Original solution set: [[-4.1206, 0.8143, -0.7775], [-3.8706, 0.8143, -0.7775], [-4.1206, 1.0643, -0.7775], [-4.1206, 0.8143, -0.5275]]


In [164]:
import time
tStart = time.time() #計時開始

# step1: Order & re-label
for j1 in range(len(S)):
    SortedS.append(S[sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][0]])
    # store best fit
    BestFit.append([S[sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][0]],
                    sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][1]])
if len(BestFit) > maxNumFit:
    del BestFit[maxNumFit:]
print('最佳組合 %s' %BestFit )
curx = SortedS[0]
minfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[0][1]
maxfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[-1][1]



# 初始溫度、溫度下限
maxT = (maxfit-minfit)/np.exp(p) # initial p = 0.9
curT = maxT
minT = 1e-5 * maxT
print('初始溫度: %.f & 終止溫度: %.f' %(curT, minT))

print('初始解(x1): %s & 初始目標函數值(f[x1]): %.f' %(curx, minfit))

#print(SortedS)
# step2
iters = 1
while ((maxfit-minfit) >= epsilon and curT > minT) or iters <= MaxIter:
    # step3
    for epoch in range(M):
        #step3.1
        k = 1
        print(SortedS)
        #step3.2
        while k <= n:
            #step3.3 Reflect
            #print(SortedS[:n-k+1])
            center = centroid(SortedS[:n-k+1])
            #print('重心:', center)
            x1 = SortedS[-k][0]; x2 = SortedS[-k][1]; x3 = SortedS[-k][2]
            ρ_ = random.uniform(0.9, 1.1) 
            #print(ρ_)
            if round(center[0] + ρ_*(center[0] - x1),4) > 5 or round(center[0] + ρ_*(center[0] - x1),4) < -5:
                x1_r = round(center[0] - ρ_*(center[0] - x1),4)
            else:
                x1_r = round(center[0] + ρ_*(center[0] - x1),4)
            if round(center[1] + ρ_*(center[1] - x2),4) > 5 or round(center[1] + ρ_*(center[1] - x2),4) < -5:
                x2_r = round(center[1] - ρ_*(center[1] - x2),4)
            else:
                x2_r = round(center[1] + ρ_*(center[1] - x2),4)  
            if round(center[2] + ρ_*(center[2] - x3),4) > 5 or round(center[2] + ρ_*(center[2] - x3),4) < -5:
                x3_r = round(center[2] - ρ_*(center[2] - x3),4)
            else:
                x3_r = round(center[2] + ρ_*(center[2] - x3),4)
                
            #print('Before Reflect :', x1, x2)
            #print('Reflected :', x1_r, x2_r)
            f_min = fitness(x1_r, x2_r, x3_r)
            #print('New fitness %.f'%f_min)
            #step3.3.1
            if f_min < minfit:
                #step3.3.3
                SortedS[-k] = [x1_r, x2_r, x3_r]
                #minfit = f_min
                # step3.5
                fit_list_1 = []
                for i2 in range(len(SortedS)):
                    x1 = SortedS[i2][0]; x2 = SortedS[i2][1]; x3 = SortedS[i2][2]
                    fit_list_1.append([i2, fitness(x1, x2, x3)])
                S_ = []    
                for j2 in range(len(SortedS)):
                    S_.append(SortedS[sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[j2][0]])
                    
                # update best fit
                print('-'*10,'Doing insert','-'*10)
                BestFit.insert(0,[[x1_r, x2_r, x3_r], f_min])
                if len(BestFit) > maxNumFit:
                    del BestFit[maxNumFit:]
                print('最佳組合 %s' %BestFit )
                
                curx = S_[0]
                SortedS = S_
                minfit = sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[0][1]
                maxfit = sorted(fit_list_1,key = lambda fit_list_1:fit_list_1[1])[-1][1]
                print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                #epoch += 1
                break
            #step3.3.2
            else:
                p = np.exp(-(f_min-minfit)/curT)
                #print('Prob: %.2f' %p)
                if p >= np.random.rand():
                    SortedS[-k] = [x1_r, x2_r, x3_r]
                    fit_list_2 = []
                    for i3 in range(len(SortedS)):
                        x1 = SortedS[i3][0]; x2 = SortedS[i3][1]; x3 = SortedS[i3][2]
                        fit_list_2.append([i3, fitness(x1, x2, x3)])
                    #print(SortedS)
                    S_ = []    
                    for j3 in range(len(SortedS)):
                        S_.append(SortedS[sorted(fit_list_2,key = lambda fit_list_2: fit_list_2[1])[j3][0]])
                    # update best fit
                    loc = 0
                    for idx in range(len(BestFit)):
                        if f_min > BestFit[idx][1]:
                            loc = idx+1
                    print('-'*10,'Doing insert','-'*10)
                    BestFit.insert(loc,[[x1_r, x2_r, x3_r], f_min])
                    if len(BestFit) > maxNumFit:
                        del BestFit[maxNumFit:]
                    print('最佳組合 %s' %BestFit )
                    
                    curx = S_[0]
                    SortedS = S_
                    minfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[0][1]
                    maxfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[-1][1]
                    print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                    #epoch += 1
                    break
                else:
                    #print('K %s' %k)
                    k += 1
            

        print('-'*10, 'end of epoch %s' %epoch, '-'*10)                            
        epoch += 1        
    curT *= alpha    
    iters += 1
    print('-'*10,'current temperature %.f & iterations %.f & fitness = %.f' %(curT, iters, minfit), '-'*10)
print('最終收斂的解與解的品質:' , curx, minfit)
tEnd = time.time()#計時結束
print ("It cost %f sec" % (tEnd - tStart))

最佳組合 [[[-0.0517, -0.0333, -0.8558], 0.7361754199999999], [[0.0057, -0.1581, -0.8488], 0.74548954], [[0.0634, -0.2836, -0.8418], 0.79307576], [[0.1161, -0.3982, -0.8354], 0.86993561], [[-0.2029, -0.6142, -0.7464], 0.9755230099999999], [[-0.1461, -0.7202, -0.742], 1.0905972499999999]]
初始溫度: 2 & 終止溫度: 0
初始解(x1): [-0.0517, -0.0333, -0.8558] & 初始目標函數值(f[x1]): 16
[[-0.0517, -0.0333, -0.8558], [0.0057, -0.1581, -0.8488], [-0.2029, -0.6142, -0.7464], [-0.9299, -0.9428, -0.7245], [-3.8706, 0.8143, -0.7775], [-4.1206, 0.8143, -0.5275], [-4.1206, 0.8143, -0.7775], [-4.1206, 1.0643, -0.7775]]
---------- Doing insert ----------
最佳組合 [[[-0.0517, -0.0333, -0.8558], 0.7361754199999999], [[0.0057, -0.1581, -0.8488], 0.74548954], [[0.0634, -0.2836, -0.8418], 0.79307576], [[0.1161, -0.3982, -0.8354], 0.86993561], [[-0.2029, -0.6142, -0.7464], 0.9755230099999999], [[-0.1461, -0.7202, -0.742], 1.0905972499999999]]
當下最佳解: [-0.0517, -0.0333, -0.8558] & fitness: 1
---------- end of epoch 0 ----------
[[-0.051

---------- end of epoch 0 ----------
[[-0.0517, -0.0333, -0.8558], [0.0057, -0.1581, -0.8488], [-0.2029, -0.6142, -0.7464], [-0.9299, -0.9428, -0.7245], [-3.8706, 0.8143, -0.7775], [3.6653, -1.5058, -0.8537], [-4.1206, 0.8143, -0.5275], [-4.1206, 0.8143, -0.7775]]
---------- end of epoch 1 ----------
[[-0.0517, -0.0333, -0.8558], [0.0057, -0.1581, -0.8488], [-0.2029, -0.6142, -0.7464], [-0.9299, -0.9428, -0.7245], [-3.8706, 0.8143, -0.7775], [3.6653, -1.5058, -0.8537], [-4.1206, 0.8143, -0.5275], [-4.1206, 0.8143, -0.7775]]
---------- end of epoch 2 ----------
---------- current temperature 0 & iterations 132 & fitness = 1 ----------
[[-0.0517, -0.0333, -0.8558], [0.0057, -0.1581, -0.8488], [-0.2029, -0.6142, -0.7464], [-0.9299, -0.9428, -0.7245], [-3.8706, 0.8143, -0.7775], [3.6653, -1.5058, -0.8537], [-4.1206, 0.8143, -0.5275], [-4.1206, 0.8143, -0.7775]]
---------- end of epoch 0 ----------
[[-0.0517, -0.0333, -0.8558], [0.0057, -0.1581, -0.8488], [-0.2029, -0.6142, -0.7464], [-0.92

In [165]:
MaxIter = 50*n
epsilon2 = 1e-6 
for idx in range(len(BestFit)):
    # Simplex Direct Search
    S = []; SortedS=[]; curx = BestFit[idx][0]
    S.append(curx)
    S.extend(simplex(curx))

    # step1: Order & re-label
    for j1 in range(len(S)):
        SortedS.append(S[sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][0]])
    #print(len(SortedS))
    curx = SortedS[0]
    minfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[0][1]
    maxfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[-1][1]

    #print('初始解(x1): %s & 初始目標函數值(f[x1]): %.f' %(curx, minfit))

    # step2
    iters = 1
    while (maxfit-minfit) >= epsilon2 and iters <= MaxIter:
        k = 1
        #print(SortedS)
        while k <= n:
            shrink = True
            #step3.3 Reflect
            #print(SortedS[:n-k+1])
            center = centroid(SortedS[:n-k+1])
            #print('重心:', center)
            x1 = SortedS[-k][0]; x2 = SortedS[-k][1]; x3 = SortedS[-k][2]
            ρ_ = random.uniform(0.9, 1.1) 
            #print(ρ_)
            if round(center[0] + ρ_*(center[0] - x1),4) > 5 or round(center[0] + ρ_*(center[0] - x1),4) < -5:
                x1_r = round(center[0] - ρ_*(center[0] - x1),4)
            else:
                x1_r = round(center[0] + ρ_*(center[0] - x1),4)
            if round(center[1] + ρ_*(center[1] - x2),4) > 5 or round(center[1] + ρ_*(center[1] - x2),4) < -5:
                x2_r = round(center[1] - ρ_*(center[1] - x2),4)
            else:
                x2_r = round(center[1] + ρ_*(center[1] - x2),4)  
            if round(center[2] + ρ_*(center[2] - x3),4) > 5 or round(center[2] + ρ_*(center[2] - x3),4) < -5:
                x3_r = round(center[2] - ρ_*(center[2] - x3),4)
            else:
                x3_r = round(center[2] + ρ_*(center[2] - x3),4)

                #print('Before Reflect :', x1, x2)
                #print('Reflected :', x1_r, x2_r)
            f_min = fitness(x1_r, x2_r, x3_r)
                #print('New fitness %.f'%f_min)
                #step3.3.1
            if f_min < minfit:
                    #step3.3.3
                SortedS[-k] = [x1_r, x2_r, x3_r]
                    #minfit = f_min
                    # step3.5
                fit_list_1 = []
                for i2 in range(len(SortedS)):
                    x1 = SortedS[i2][0]; x2 = SortedS[i2][1]; x3 = SortedS[i2][2]
                    fit_list_1.append([i2, fitness(x1, x2, x3)])
                S_ = []    
                for j2 in range(len(SortedS)):
                    S_.append(SortedS[sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[j2][0]])
                curx = S_[0]
                SortedS = S_
                minfit = sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[0][1]
                maxfit = sorted(fit_list_1,key = lambda fit_list_1:fit_list_1[1])[-1][1]
                #print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                shrink = False
                iters += 1
                break
                #step3.3.2
            else:
                p = np.exp(-(f_min-minfit)/curT)
                #print('Prob: %.2f' %p)
                if p >= np.random.rand():
                    SortedS[-k] = [x1_r, x2_r, x3_r]
                    fit_list_2 = []
                    for i3 in range(len(SortedS)):
                        x1 = SortedS[i3][0]; x2 = SortedS[i3][1]; x3 = SortedS[i3][2]
                        fit_list_2.append([i3, fitness(x1, x2, x3)])
                    #print(SortedS)
                    S_ = []    
                    for j3 in range(len(SortedS)):
                        S_.append(SortedS[sorted(fit_list_2,key = lambda fit_list_2: fit_list_2[1])[j3][0]])
                    curx = S_[0]
                    SortedS = S_
                    minfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[0][1]
                    maxfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[-1][1]
                    #print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                    shrink = False
                    iters += 1
                    break
                else:
                    #print('K %s' %k)
                    k += 1


        # shrink
        if shrink:            
            #print('Before shrink %s' % SortedS)
            S = [SortedS[0]]
            for val in range(1,n+1):
                x1 = SortedS[-val][0]; x2 = SortedS[-val][1]; x3 = SortedS[-val][2]
                x1_s = round(SortedS[0][0] + sigma*(x1 - SortedS[0][0]), 4)
                x2_s = round(SortedS[0][1] + sigma*(x2 - SortedS[0][1]), 4)
                x3_s = round(SortedS[0][2] + sigma*(x3 - SortedS[0][2]), 4)
                S.append([x1_s, x2_s, x3_s])

            #print('After shrink %s' % S)

                # 計算 排序
            fit_list_3 = []
            for i4 in range(len(S)):
                x1 = S[i4][0]; x2 = S[i4][1]; x3 = S[i4][2]
                fit_list_3.append([i4, fitness(x1, x2, x3)])
            SortedS = []    
            for j4 in range(len(S)):
                SortedS.append(S[sorted(fit_list_3,key = lambda fit_list_3: fit_list_3[1])[j4][0]])
            curx = SortedS[0]
            minfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[0][1]
            maxfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[-1][1]
            #print('當下最佳解: %s & fitness: %.f' %(curx, minfit))


        #print('-'*10, 'end of iteration %s' %iters, '-'*10)                                
        iters += 1
    print('最終收斂的解與解的品質:' , curx, minfit)


最終收斂的解與解的品質: [-0.0008, 0.0002, 0.0007] 1.17e-06
最終收斂的解與解的品質: [-0.0001, -0.0001, 0.0004] 1.8e-07
最終收斂的解與解的品質: [-0.0809, 0.0638, 0.1244] 0.02609061
最終收斂的解與解的品質: [0.0139, 0.0151, 0.0095] 0.00051147
最終收斂的解與解的品質: [-0.0284, -0.1969, 0.2775] 0.11658242
最終收斂的解與解的品質: [-0.0331, 0.0248, 0.0109] 0.0018294599999999998


## Colville function (CV )

In [166]:
def fitness(x1, x2, x3, x4):
    return 100*(x1**2-x2)**2+(x1-1)**2+(x3-1)**2+90*(x3**2-x4)**2+10.1*((x2-1)**2+(x4-1)**2)+19.8*(x2-1)*(x4-1)

def simplex3(init_sol, n = n, edge_length = 5):
    sx_sol = []
    if init_sol[0] + edge_length > 5:
        sx_sol.append([round(init_sol[0] - edge_length, 4), init_sol[1], init_sol[2], init_sol[3]])
    else:
        sx_sol.append([round(init_sol[0] + edge_length, 4), init_sol[1], init_sol[2], init_sol[3]])
    if init_sol[1] + edge_length > 5:
        sx_sol.append([init_sol[0], round(init_sol[1] - edge_length, 4), init_sol[2], init_sol[3]])
    else:
        sx_sol.append([init_sol[0], round(init_sol[1] + edge_length, 4), init_sol[2], init_sol[3]])
    if init_sol[2] + edge_length > 5:
        sx_sol.append([init_sol[0], init_sol[1], round(init_sol[2] - edge_length, 4), init_sol[3]])
    else:
        sx_sol.append([init_sol[0], init_sol[1], round(init_sol[2] + edge_length, 4), init_sol[3]])
    if init_sol[3] + edge_length > 5:
        sx_sol.append([init_sol[0], init_sol[1], init_sol[2], round(init_sol[3] - edge_length, 4)])
    else:
        sx_sol.append([init_sol[0], init_sol[1], init_sol[2], round(init_sol[3] + edge_length, 4)])
    return sx_sol

def centroid(dataset):
    x1_sum = 0; x2_sum = 0; x3_sum = 0; x4_sum = 0
    for i in range(len(dataset)):
        x1_sum += dataset[i][0] 
        x2_sum += dataset[i][1]
        x3_sum += dataset[i][2]
        x4_sum += dataset[i][3]
    return round(x1_sum/(len(dataset)), 4), round(x2_sum/(len(dataset)), 4), round(x3_sum/(len(dataset)), 4), round(x4_sum/(len(dataset)), 4)

In [174]:
# step0 Setting parameters
np.random.seed(11)
# 變數數量、每個溫度迭代次數

n = 4
M = n
MaxIter = 50*n

# sigma & accepted probability & termination criteria
sigma = np.random.rand()
p = 0.9                    #initial
epsilon = 1e-8 

# 溫度衰減係數
alpha = 0.5 # 調低 加速降溫

# 初始解
curx = [round(random.uniform(-10, 10), 4), round(random.uniform(-10, 10), 4),
        round(random.uniform(-10, 10), 4), round(random.uniform(-10, 10), 4)]
print("初始解：", curx)

# BEST Fit
BestFit = []
maxNumFit = 2*n

# Simplex Direct Search
S = []; SortedS=[]
S.append(curx)
S.extend(simplex3(curx, edge_length = 0.25))
print("Original solution set: %s" %S)
fit_list = []
for i1 in range(len(S)):
    x1 = S[i1][0]; x2 = S[i1][1]; x3 = S[i1][2]; x4 = S[i1][3]
    fit_list.append([i1, fitness(x1, x2, x3, x4)])

初始解： [5.4377, -1.8699, 0.8913, -6.3199]
Original solution set: [[5.4377, -1.8699, 0.8913, -6.3199], [5.1877, -1.8699, 0.8913, -6.3199], [5.4377, -1.6199, 0.8913, -6.3199], [5.4377, -1.8699, 1.1413, -6.3199], [5.4377, -1.8699, 0.8913, -6.0699]]


In [175]:
import time
tStart = time.time() #計時開始

# step1: Order & re-label
for j1 in range(len(S)):
    SortedS.append(S[sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][0]])
    # store best fit
    BestFit.append([S[sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][0]],
                    sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][1]])
if len(BestFit) > maxNumFit:
    del BestFit[maxNumFit:]
print('最佳組合 %s' %BestFit )
curx = SortedS[0]
minfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[0][1]
maxfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[-1][1]



# 初始溫度、溫度下限
maxT = (maxfit-minfit)/np.exp(p) # initial p = 0.9
curT = maxT
minT = 1e-5 * maxT
print('初始溫度: %.f & 終止溫度: %.f' %(curT, minT))

print('初始解(x1): %s & 初始目標函數值(f[x1]): %.f' %(curx, minfit))

#print(SortedS)
# step2
iters = 1
while ((maxfit-minfit) >= epsilon and curT > minT) or iters <= MaxIter:
    # step3
    for epoch in range(M):
        #step3.1
        k = 1
        print(SortedS)
        #step3.2
        while k <= n:
            #step3.3 Reflect
            #print(SortedS[:n-k+1])
            center = centroid(SortedS[:n-k+1])
            #print('重心:', center)
            x1 = SortedS[-k][0]; x2 = SortedS[-k][1]; x3 = SortedS[-k][2]; x4 = SortedS[-k][3]
            ρ_ = random.uniform(0.9, 1.1) 
            #print(ρ_)
            ub = 2; lb = -2
            if round(center[0] + ρ_*(center[0] - x1),4) > ub or round(center[0] + ρ_*(center[0] - x1),4) < lb:
                x1_r = round(center[0] - ρ_*(center[0] - x1),4)
            else:
                x1_r = round(center[0] + ρ_*(center[0] - x1),4)
            if round(center[1] + ρ_*(center[1] - x2),4) > ub or round(center[1] + ρ_*(center[1] - x2),4) < lb:
                x2_r = round(center[1] - ρ_*(center[1] - x2),4)
            else:
                x2_r = round(center[1] + ρ_*(center[1] - x2),4)  
            if round(center[2] + ρ_*(center[2] - x3),4) > ub or round(center[2] + ρ_*(center[2] - x3),4) < lb:
                x3_r = round(center[2] - ρ_*(center[2] - x3),4)
            else:
                x3_r = round(center[2] + ρ_*(center[2] - x3),4)
            if round(center[3] + ρ_*(center[3] - x4),4) > ub or round(center[3] + ρ_*(center[3] - x4),4) < lb:
                x4_r = round(center[3] - ρ_*(center[3] - x4),4)
            else:
                x4_r = round(center[3] + ρ_*(center[3] - x4),4)    
            #print('Before Reflect :', x1, x2)
            #print('Reflected :', x1_r, x2_r)
            f_min = fitness(x1_r, x2_r, x3_r, x4_r)
            #print('New fitness %.f'%f_min)
            #step3.3.1
            if f_min < minfit:
                #step3.3.3
                SortedS[-k] = [x1_r, x2_r, x3_r, x4_r]
                #minfit = f_min
                # step3.5
                fit_list_1 = []
                for i2 in range(len(SortedS)):
                    x1 = SortedS[i2][0]; x2 = SortedS[i2][1]; x3 = SortedS[i2][2]; x4 = SortedS[i2][3]
                    fit_list_1.append([i2, fitness(x1, x2, x3, x4)])
                S_ = []    
                for j2 in range(len(SortedS)):
                    S_.append(SortedS[sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[j2][0]])
                    
                # update best fit
                print('-'*10,'Doing insert','-'*10)
                BestFit.insert(0,[[x1_r, x2_r, x3_r, x4_r], f_min])
                if len(BestFit) > maxNumFit:
                    del BestFit[maxNumFit:]
                print('最佳組合 %s' %BestFit )
                
                curx = S_[0]
                SortedS = S_
                minfit = sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[0][1]
                maxfit = sorted(fit_list_1,key = lambda fit_list_1:fit_list_1[1])[-1][1]
                print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                #epoch += 1
                break
            #step3.3.2
            else:
                p = np.exp(-(f_min-minfit)/curT)
                #print('Prob: %.2f' %p)
                if p >= np.random.rand():
                    SortedS[-k] = [x1_r, x2_r, x3_r, x4_r]
                    fit_list_2 = []
                    for i3 in range(len(SortedS)):
                        x1 = SortedS[i3][0]; x2 = SortedS[i3][1]; x3 = SortedS[i3][2]; x4 = SortedS[i3][3]
                        fit_list_2.append([i3, fitness(x1, x2, x3, x4)])
                    #print(SortedS)
                    S_ = []    
                    for j3 in range(len(SortedS)):
                        S_.append(SortedS[sorted(fit_list_2,key = lambda fit_list_2: fit_list_2[1])[j3][0]])
                    # update best fit
                    loc = 0
                    for idx in range(len(BestFit)):
                        if f_min > BestFit[idx][1]:
                            loc = idx+1
                    print('-'*10,'Doing insert','-'*10)
                    BestFit.insert(loc,[[x1_r, x2_r, x3_r, x4_r], f_min])
                    if len(BestFit) > maxNumFit:
                        del BestFit[maxNumFit:]
                    print('最佳組合 %s' %BestFit )
                    
                    curx = S_[0]
                    SortedS = S_
                    minfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[0][1]
                    maxfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[-1][1]
                    print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                    #epoch += 1
                    break
                else:
                    #print('K %s' %k)
                    k += 1
            

        print('-'*10, 'end of epoch %s' %epoch, '-'*10)                            
        epoch += 1        
    curT *= alpha    
    iters += 1
    print('-'*10,'current temperature %.f & iterations %.f & fitness = %.f' %(curT, iters, minfit), '-'*10)
print('最終收斂的解與解的品質:' , curx, minfit)
tEnd = time.time()#計時結束
print ("It cost %f sec" % (tEnd - tStart))

最佳組合 [[[5.1877, -1.8699, 0.8913, -6.3199], 88454.17119318795], [[5.4377, -1.6199, 0.8913, -6.3199], 102837.26065137626], [[5.4377, -1.8699, 0.8913, -6.0699], 104087.97050982625], [[5.4377, -1.8699, 0.8913, -6.3199], 104453.02996587625], [[5.4377, -1.8699, 1.1413, -6.3199], 105127.00270711847]]
初始溫度: 6779 & 終止溫度: 0
初始解(x1): [5.1877, -1.8699, 0.8913, -6.3199] & 初始目標函數值(f[x1]): 88454
[[5.1877, -1.8699, 0.8913, -6.3199], [5.4377, -1.6199, 0.8913, -6.3199], [5.4377, -1.8699, 0.8913, -6.0699], [5.4377, -1.8699, 0.8913, -6.3199], [5.4377, -1.8699, 1.1413, -6.3199]]
---------- Doing insert ----------
最佳組合 [[[5.1877, -1.8699, 0.8913, -6.3199], 88454.17119318795], [[5.4377, -1.6199, 0.8913, -6.3199], 102837.26065137626], [[5.442, -1.7406, 0.6239, -6.3242], 103410.96202485017], [[5.4377, -1.8699, 0.8913, -6.0699], 104087.97050982625], [[5.4377, -1.8699, 0.8913, -6.3199], 104453.02996587625], [[5.4377, -1.8699, 1.1413, -6.3199], 105127.00270711847]]
當下最佳解: [5.1877, -1.8699, 0.8913, -6.3199] & fitn

---------- end of epoch 3 ----------
---------- current temperature 0 & iterations 101 & fitness = 88454 ----------
[[5.1877, -1.8699, 0.8913, -6.3199], [5.4377, -1.6199, 0.8913, -6.3199], [5.4431, -1.6719, 0.7516, -6.3253], [5.442, -1.7406, 0.6239, -6.3242], [5.4377, -1.8699, 0.8913, -6.0699]]
---------- end of epoch 0 ----------
[[5.1877, -1.8699, 0.8913, -6.3199], [5.4377, -1.6199, 0.8913, -6.3199], [5.4431, -1.6719, 0.7516, -6.3253], [5.442, -1.7406, 0.6239, -6.3242], [5.4377, -1.8699, 0.8913, -6.0699]]
---------- end of epoch 1 ----------
[[5.1877, -1.8699, 0.8913, -6.3199], [5.4377, -1.6199, 0.8913, -6.3199], [5.4431, -1.6719, 0.7516, -6.3253], [5.442, -1.7406, 0.6239, -6.3242], [5.4377, -1.8699, 0.8913, -6.0699]]
---------- end of epoch 2 ----------
[[5.1877, -1.8699, 0.8913, -6.3199], [5.4377, -1.6199, 0.8913, -6.3199], [5.4431, -1.6719, 0.7516, -6.3253], [5.442, -1.7406, 0.6239, -6.3242], [5.4377, -1.8699, 0.8913, -6.0699]]
---------- end of epoch 3 ----------
---------- curre

---------- end of epoch 3 ----------
---------- current temperature 0 & iterations 180 & fitness = 88454 ----------
[[5.1877, -1.8699, 0.8913, -6.3199], [5.4377, -1.6199, 0.8913, -6.3199], [5.4431, -1.6719, 0.7516, -6.3253], [5.442, -1.7406, 0.6239, -6.3242], [5.4377, -1.8699, 0.8913, -6.0699]]
---------- end of epoch 0 ----------
[[5.1877, -1.8699, 0.8913, -6.3199], [5.4377, -1.6199, 0.8913, -6.3199], [5.4431, -1.6719, 0.7516, -6.3253], [5.442, -1.7406, 0.6239, -6.3242], [5.4377, -1.8699, 0.8913, -6.0699]]
---------- end of epoch 1 ----------
[[5.1877, -1.8699, 0.8913, -6.3199], [5.4377, -1.6199, 0.8913, -6.3199], [5.4431, -1.6719, 0.7516, -6.3253], [5.442, -1.7406, 0.6239, -6.3242], [5.4377, -1.8699, 0.8913, -6.0699]]
---------- end of epoch 2 ----------
[[5.1877, -1.8699, 0.8913, -6.3199], [5.4377, -1.6199, 0.8913, -6.3199], [5.4431, -1.6719, 0.7516, -6.3253], [5.442, -1.7406, 0.6239, -6.3242], [5.4377, -1.8699, 0.8913, -6.0699]]
---------- end of epoch 3 ----------
---------- curre

In [176]:
MaxIter = 50*n
epsilon2 = 1e-6 
for idx in range(len(BestFit)):
    # Simplex Direct Search
    S = []; SortedS=[]; curx = BestFit[idx][0]
    S.append(curx)
    S.extend(simplex3(curx,edge_length = 0.125))
    # step1: Order & re-label
    for j1 in range(len(S)):
        SortedS.append(S[sorted(fit_list,key = lambda fit_list: fit_list[1])[j1][0]])
    #print(len(SortedS))
    curx = SortedS[0]
    minfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[0][1]
    maxfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[-1][1]

    #print('初始解(x1): %s & 初始目標函數值(f[x1]): %.f' %(curx, minfit))
    #print(SortedS)

    # step2
    iters = 1
    while (maxfit-minfit) >= epsilon2 and iters <= MaxIter:
        k = 1
        #print(SortedS)
        while k <= n:
            shrink = True
            #step3.3 Reflect
            #print(SortedS[:n-k+1])
            center = centroid(SortedS[:n-k+1])
            #print('重心:', center)
            x1 = SortedS[-k][0]; x2 = SortedS[-k][1]; x3 = SortedS[-k][2]; x4 = SortedS[-k][3]
            ρ_ = random.uniform(0.9, 1.1) 
            #print(ρ_)
            if round(center[0] + ρ_*(center[0] - x1),4) > 2 or round(center[0] + ρ_*(center[0] - x1),4) < -2:
                x1_r = round(center[0] - ρ_*(center[0] - x1),4)
            else:
                x1_r = round(center[0] + ρ_*(center[0] - x1),4)
            if round(center[1] + ρ_*(center[1] - x2),4) > 2 or round(center[1] + ρ_*(center[1] - x2),4) < -2:
                x2_r = round(center[1] - ρ_*(center[1] - x2),4)
            else:
                x2_r = round(center[1] + ρ_*(center[1] - x2),4)  
            if round(center[2] + ρ_*(center[2] - x3),4) > 2 or round(center[2] + ρ_*(center[2] - x3),4) < -2:
                x3_r = round(center[2] - ρ_*(center[2] - x3),4)
            else:
                x3_r = round(center[2] + ρ_*(center[2] - x3),4)
            if round(center[3] + ρ_*(center[3] - x4),4) > 2 or round(center[3] + ρ_*(center[3] - x4),4) < -2:
                x4_r = round(center[3] - ρ_*(center[3] - x4),4)
            else:
                x4_r = round(center[3] + ρ_*(center[3] - x4),4)
                #print('Before Reflect :', x1, x2)
                #print('Reflected :', x1_r, x2_r)
            f_min = fitness(x1_r, x2_r, x3_r, x4_r)
                #print('New fitness %.f'%f_min)
                #step3.3.1
            if f_min < minfit:
                    #step3.3.3
                SortedS[-k] = [x1_r, x2_r, x3_r, x4_r]
                    #minfit = f_min
                    # step3.5
                fit_list_1 = []
                for i2 in range(len(SortedS)):
                    x1 = SortedS[i2][0]; x2 = SortedS[i2][1]; x3 = SortedS[i2][2]; x4 = SortedS[i2][3]
                    fit_list_1.append([i2, fitness(x1, x2, x3, x4)])
                S_ = []    
                for j2 in range(len(SortedS)):
                    S_.append(SortedS[sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[j2][0]])
                curx = S_[0]
                SortedS = S_
                minfit = sorted(fit_list_1,key = lambda fit_list_1: fit_list_1[1])[0][1]
                maxfit = sorted(fit_list_1,key = lambda fit_list_1:fit_list_1[1])[-1][1]
                #print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                shrink = False
                iters += 1
                break
                #step3.3.2
            else:
                p = np.exp(-(f_min-minfit)/curT)
                #print('Prob: %.2f' %p)
                if p >= np.random.rand():
                    SortedS[-k] = [x1_r, x2_r, x3_r, x4_r]
                    fit_list_2 = []
                    for i3 in range(len(SortedS)):
                        x1 = SortedS[i3][0]; x2 = SortedS[i3][1]; x3 = SortedS[i3][2]; x4 = SortedS[i3][3]
                        fit_list_2.append([i3, fitness(x1, x2, x3, x4)])
                    #print(SortedS)
                    S_ = []    
                    for j3 in range(len(SortedS)):
                        S_.append(SortedS[sorted(fit_list_2,key = lambda fit_list_2: fit_list_2[1])[j3][0]])
                    curx = S_[0]
                    SortedS = S_
                    minfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[0][1]
                    maxfit = sorted(fit_list_2,key = lambda fit_list: fit_list_2[1])[-1][1]
                    #print('當下最佳解: %s & fitness: %.f' %(curx, minfit))
                    shrink = False
                    iters += 1
                    break
                else:
                    #print('K %s' %k)
                    k += 1


        # shrink
        if shrink:            
            #print('Before shrink %s' % SortedS)
            S = [SortedS[0]]
            for val in range(1,n+1):
                x1 = SortedS[-val][0]; x2 = SortedS[-val][1]; x3 = SortedS[-val][2]; x4 = SortedS[-val][3]
                x1_s = round(SortedS[0][0] + sigma*(x1 - SortedS[0][0]), 4)
                x2_s = round(SortedS[0][1] + sigma*(x2 - SortedS[0][1]), 4)
                x3_s = round(SortedS[0][2] + sigma*(x3 - SortedS[0][2]), 4)
                x4_s = round(SortedS[0][3] + sigma*(x4 - SortedS[0][3]), 4)
                S.append([x1_s, x2_s, x3_s, x4_s])

            #print('After shrink %s' % S)

                # 計算 排序
            fit_list_3 = []
            for i4 in range(len(S)):
                x1 = S[i4][0]; x2 = S[i4][1]; x3 = S[i4][2]; x4 = S[i4][3]
                fit_list_3.append([i4, fitness(x1, x2, x3, x4)])
            SortedS = []    
            for j4 in range(len(S)):
                SortedS.append(S[sorted(fit_list_3,key = lambda fit_list_3: fit_list_3[1])[j4][0]])
            curx = SortedS[0]
            minfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[0][1]
            maxfit = sorted(fit_list,key = lambda fit_list: fit_list[1])[-1][1]
            #print('當下最佳解: %s & fitness: %.f' %(curx, minfit))


        #print('-'*10, 'end of iteration %s' %iters, '-'*10)                                
        iters += 1
    print('最終收斂的解與解的品質:' , curx, minfit)


最終收斂的解與解的品質: [5.0627, -1.8699, 0.8913, -6.3199] 81241.60389781254
最終收斂的解與解的品質: [5.3127, -1.6199, 0.8913, -6.3199] 88454.17119318795
最終收斂的解與解的品質: [5.3181, -1.6719, 0.7516, -6.3253] 88454.17119318795
最終收斂的解與解的品質: [5.317, -1.7406, 0.6239, -6.3242] 88454.17119318795
最終收斂的解與解的品質: [5.3127, -1.8699, 0.8913, -6.0699] 88454.17119318795
最終收斂的解與解的品質: [5.3127, -1.8699, 0.8913, -6.3199] 88454.17119318795
最終收斂的解與解的品質: [5.3127, -1.8699, 1.1413, -6.3199] 88454.17119318795
